In [39]:
import jieba

# 基本聊天机器人常识

In [40]:
# 全模式
result = jieba.lcut("下雨天留客天留我不留", cut_all=True)
print("精确分词模式：",result)

精确分词模式： ['下雨', '下雨天', '雨天', '留客', '天', '留', '我', '不留']


In [41]:
# 精确模式
result = jieba.lcut("下雨天留客天留我不留", cut_all=False)
print("全模式：",result)

全模式： ['下雨天', '留客', '天留', '我', '不留']


In [42]:
# 搜索引擎模式
result = jieba.lcut_for_search("下雨天留客天留我不留")
print("搜索引擎模式：",result)

搜索引擎模式： ['下雨', '雨天', '下雨天', '留客', '天留', '我', '不留']


# 文本表示

In [43]:
# 定义词转向量方式
def get_word_vector_result(word):
    result = []
    for i in word_vector_list:
        if i == word:
            result.append(1)
        else:
            result.append(0)
    return result

In [44]:
word_vector_list = ["下雨天","留客","天留","我","不留"]
word = "下雨天留客天留我不留"
result = jieba.lcut(word)
print("分词结果：",result)
word_vector = []
for i in result:
    word_vector.append(get_word_vector_result(i))
print("词向量结果：",word_vector)

分词结果： ['下雨天', '留客', '天留', '我', '不留']
词向量结果： [[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]]


# 项目实现

In [45]:
import pandas as pd

## 语料库处理

In [67]:
f1 = pd.read_csv("taozhe.txt", encoding="utf-8")


In [66]:
# 对 语 料 库 进 行 分 词 ，分 词 结 果 存 人 f e n c i _ r e s u l t . t x t 文 件 
f1 = open ('taozhe.txt', encoding="utf8")
f2 = open ('fenci_result.txt', 'a', encoding= "utf8") 
lines = f1.readlines()
for line in lines:
    line.replace('\t', '').replace ('\n', '') .replace(' ', '')
    seg_list = jieba. cut(line) # 将词汇以空格隔开
    f2.write(' '.join(seg_list)) 
f1.close()
f2.close()

In [69]:
from gensim.models import word2vec
sentences = word2vec.Text8Corpus("fenci_result.txt")
model = word2vec.Word2Vec(sentences)
model.save("word2vec.model")

## 对话处理

In [98]:
import json
f3 = open('templet.json',  encoding='utf-8')
str = ''
for line in f3.readlines():
    str += line
contents = json.loads(str)
f3.close()
print(contents)

[{'title': '陶喆的人机交往', 'reply': '啊啊啊南宁的朋友 胡彦斌我好想你 penny我做了错饭 我不喜欢程度因为我结婚了因为成都美女太多了'}, {'title': '陶喆的歌曲', 'reply': '月亮代表谁的心 小镇姑娘 流沙 十七岁 爱我还是她 飞机场的10:30'}]


## 寻找最大相似度回答

In [107]:
for i in range(len(contents)):
    print(contents[i]['title'])

陶喆的人机交往
陶喆的歌曲


In [108]:
import random
import math
def answer(input):
    similarityMax =0
    similarityIndex = -1

    input_word_arr = list(jieba.cut(input))

    #遍历规则库

    for i in range(len(contents)):
        title_word_arr = list(jieba.cut(contents[i]['title'].replace(' ','')))
        try:
            similarity = model.wv.n_similarity(input_word_arr,title_word_arr)
        except Exception:
            similarity =0

        if similarityMax < similarity:
            similarityMax = similarity
            similarityIndex = i
    
    if similarityIndex != -1:
        # 随机返回一个回答
        reply_index = math.floor(random.random() * len(contents[similarityIndex]['reply']))
        if reply_index>=0:
            return {'title':contents[similarityIndex]['title'],"reply": contents[similarityIndex]['reply'][reply_index]}
    
    return {"title":"无", "reply":"抱歉我不太明白你的意思"}

In [109]:
my_input = input("请输入：")
print(answer(my_input))

{'title': '陶喆的歌曲', 'reply': ' '}
